In [1]:
import os

os.environ['KERAS_BACKEND'] = "torch"

## Regularization
>We hope to learn vector representations of the most representative aspects for a review dataset.
However, the aspect embedding matrix T may suffer from redundancy problems during training. [...] 
> The regularization term encourages orthogonality among the rows of the aspect embedding matrix T and penalizes redundancy between different aspect vectors
> ~ Ruidan

We use an Orthogonal Regulizer definition of the method can be found here: https://paperswithcode.com/method/orthogonal-regularization. <br/>
For the code we use the default implementation provided by Keras (https://keras.io/api/layers/regularizers/)

In [219]:
from keras import ops as K
from keras import backend as B


def ortho_reg(W):
    ### Orthogonal regularization for aspect embedding matrix by Ruidan     ###
    w_n = W / K.cast(B.epsilon() + K.sqrt(K.sum(K.square(W), axis=-1, keepdims=True)), B.floatx())
    # sum(w_n * w_n_t - I) * factor
    return K.sum(K.square(K.dot(w_n, K.transpose(w_n)) - K.eye(w_n.shape[0])))

In [ ]:
# TODO: Sarebbe da vedere come l'implementazione di Ruidan sia diversa da quella di Keras. Se effettivamente questa importa. A guardare i numeri sono effettivamente diversi!

In [2]:
corpus_file = "./../data/corpus.preprocessed.csv"  # It's this
# TODO GET MAXLEN FROM EMBEDDINGS DATASET (Which is input shape)
input_shape = (256,)

## Model Setup

In [3]:
import model.embeddings as embeddings

embeddings_model = embeddings.WordEmbedding(
    embeddings.LoadCorpusUtility(), max_vocab_size=10000, embedding_size=128,
    target_model_file="./../data/word-embeddings.model", corpus_file=corpus_file
)
aspect_embeddings_model = embeddings.AspectEmbedding(
    aspect_size=4, embedding_size=128, base_embeddings=embeddings_model,
    target_model_file="./../data/aspects-embedding.model"
)

/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/jacopo/PycharmProjects/nlp-course-project/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
embeddings_model.load_model()
aspect_embeddings_model.load_model()

In [5]:
from model.model import ABAEModelGenerator

generator = ABAEModelGenerator(input_shape, embeddings_model, aspect_embeddings_model)
model = generator.make_model()

ValueError: Exception encountered when calling WeightedSumLayer.call().

[1mCould not automatically infer the output shape / dtype of 'weighted_sum_layer' (of type WeightedSumLayer). Either the `WeightedSumLayer.call()` method is incorrect, or you need to implement the `WeightedSumLayer.compute_output_spec() / compute_output_shape()` method. Error encountered:

too many values to unpack (expected 2)[0m

Arguments received by WeightedSumLayer.call():
  • args=('<KerasTensor shape=(None, 256, 128), dtype=float32, sparse=False, name=keras_tensor>', '<KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_3>')
  • kwargs=<class 'inspect._empty'>

#### Load the data

## Train